# Data

In [ ]:
!pip install pyvi
!pip install transformers
!pip install underthesea
!pip install sentence_transformers
!pip install pandarallel
# !pip install datasets


In [ ]:
import json
import torch
import pandas as pd
from tqdm import tqdm
from pyvi import ViTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
import pickle
from sentence_transformers import SentenceTransformer, util
tqdm.pandas()
import json
import re
from transformers import pipeline
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
from scipy import sparse
import sklearn
import underthesea
import os
from glob import glob
from pprint import pprint
import nltk
from underthesea import word_tokenize
from underthesea import text_normalize
from sklearn.decomposition import TruncatedSVD
import math
import torch

In [ ]:
!wget https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
!wget https://dl-challenge.zalo.ai/e2e-question-answering/wikipedia_20220620_cleaned.zip
!wget https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt
!unzip -q /content/e2eqa-train+public_test-v1.zip
!unzip -q /content/wikipedia_20220620_cleaned.zip

--2022-11-28 13:59:33--  https://dl-challenge.zalo.ai/e2e-question-answering/e2eqa-train+public_test-v1.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3950125 (3.8M) [application/zip]
Saving to: ‘e2eqa-train+public_test-v1.zip’

e2eqa-train+public_ 100%[===================>]   3.77M  1.97MB/s    in 1.9s    

2022-11-28 13:59:36 (1.97 MB/s) - ‘e2eqa-train+public_test-v1.zip’ saved [3950125/3950125]



In [ ]:
data_file = open('e2eqa-train+public_test-v1/zac2022_train_merged_final.json')
data = json.load(data_file)
stopword_file = open('vietnamese-stopwords-dash.txt', 'r')
stopword = stopword_file.read()
stopword = stopword.split('\n')

### ModelForQuestionAnswering

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("nguyenvulebinh/vi-mrc-large")
model = AutoModelForQuestionAnswering.from_pretrained("nguyenvulebinh/vi-mrc-large")

qa_model = pipeline("question-answering", model=model,tokenizer=tokenizer,device=0)


# "hogger32/xlmRoberta-for-VietnameseQA", "nguyenvulebinh/vi-mrc-base"

In [ ]:
def remove_stopword(lsword):
  return lsword
  res = []
  for word in lsword.split():
    if not word in stopword:
      res.append(word) 
  return res

In [ ]:
# model_encode_query = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model_encode_query = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

### original corpus encoding to tensor

In [ ]:
!gdown 1u1wTTfPFZbHpTgq-IdeomRRPOnuoYkBR      #corpus_pkl

Downloading...
From: https://drive.google.com/uc?id=1u1wTTfPFZbHpTgq-IdeomRRPOnuoYkBR
To: /content/corpus_embeddings.pkl
100% 1.96G/1.96G [00:27<00:00, 71.4MB/s]


In [ ]:
corpus_embeddings = pickle.load(open('/content/corpus_embeddings.pkl','rb'))

### corpus after spliting and encoding to tensor

In [ ]:
# !gdown 1XuJ_mBQaTnzcTmslYmyFBKzGjjjxRfyR
# !gdown 1mJuDdWzLx73UBIfenxNiPmGJD9h8ohcY
# !gdown 1l_OMRTJ3yURj5roce7WNSajupkkSDeFy
# !gdown 1p_2pi3Ac5Z3aIewI1vG4cqIz39U2tgZH
# !gdown 1vxJ3OD1CpLCzuRD777pB935wiZE-xvpc
# !gdown 1KZhIJ61DdcMt5nj7bbTM-RFQYEGxkKpS
# !gdown 1K-TVOe1YGMb-F7cd2vlJzkuYNeAQczmt
# !gdown 1YPRjMuKH3-7qAJ5RYqz1APKeF-cRTlIq
# new_df11=pd.read_csv("new_df11.csv")
# new_df12=pd.read_csv("new_df12.csv")
# new_df21=pd.read_csv("new_df21.csv")
# new_df22=pd.read_csv("new_df22.csv")
# new=pd.concat([new_df11, new_df12,new_df21,new_df22], ignore_index=True)
# corpus_embeddings_392227 = pickle.load(open('passage_corpus_embeddings_392227.pkl','rb'))
# corpus_embeddings_784454 = pickle.load(open('passage_corpus_embeddings_784454.pkl','rb'))
# corpus_embeddings_1176681 = pickle.load(open('passage_corpus_embeddings_1176681.pkl','rb'))
# corpus_embeddings_1568909 = pickle.load(open('passage_corpus_embeddings_1568909.pkl','rb'))
# corpus_embeddings_split= torch.cat((corpus_embeddings_392227, corpus_embeddings_784454,
#                               corpus_embeddings_1176681,corpus_embeddings_1568909), 0)

### Corpus split

In [ ]:
# !gdown 1S1GXORdEP2nsFkPX6d_qzTu-Z4Q8RaAk
import pandas as pd
df=pd.read_csv('/content/wiki.csv')

Downloading...
From: https://drive.google.com/uc?id=1S1GXORdEP2nsFkPX6d_qzTu-Z4Q8RaAk
To: /content/wiki.csv
100% 1.45G/1.45G [00:09<00:00, 146MB/s]


In [ ]:
df.head()

,Unnamed: 0,id,url,title,text,timestamp,revid
0,0,2,https://vi.wikipedia.org/wiki?curid=2,Trang Chính,Trang Chính. __NOEDITSECTION__.,2022-05-12 12:46:53+00:00,68591979
1,1,4,https://vi.wikipedia.org/wiki?curid=4,Internet Society,Internet Society. Internet Society hay ISOC là...,2022-01-20 07:59:10+00:00,67988747
2,2,13,https://vi.wikipedia.org/wiki?curid=13,Tiếng Việt,"Tiếng Việt. Tiếng Việt, cũng gọi là tiếng Việt...",2022-05-29 03:42:42+00:00,68660631
3,3,24,https://vi.wikipedia.org/wiki?curid=24,Ohio,"Ohio. Ohio (viết tắt là OH, viết tắt cũ là O.)...",2022-04-17 08:15:22+00:00,68482118
4,4,26,https://vi.wikipedia.org/wiki?curid=26,California,"California. California (phát âm như ""Ca-li-phó...",2022-06-16 15:27:07+00:00,68738039


In [ ]:
def segment_documents(data,docs, max_doc_length=300):
  # List containing full and segmented docs
  segmented_docs = []
  new_df = pd.DataFrame(columns=['title','text'])
  i=0
  for doc in tqdm(docs):

    # Split document by spaces to obtain a word count that roughly approximates the token count
    split_to_words = doc.split(" ")

    # If the document is longer than our maximum length, split it up into smaller segments and add them to the list 
    if len(split_to_words) > max_doc_length:
      for doc_segment in range(0, len(split_to_words), max_doc_length):
        a= " ".join(split_to_words[doc_segment:doc_segment + max_doc_length])
        # segmented_docs.append(a)
        # temp={'id':data.iloc[i]['id'],'title':data.iloc[i]['title'],'text':a}
        new_df.loc[len(new_df.index)] = [data.iloc[i]['title'], a]


    # If the document is shorter than our maximum length, add it to the list
    else:
      # segmented_docs.append(doc)
      new_df.loc[len(new_df.index)] = [data.iloc[i]['title'], doc]
      # temp={'id':data.iloc[i]['id'],'title':data.iloc[i]['title'],'text':doc}
      # new_df=new_df.append(temp,ignore_index=True)
    i+=1
    # print('i',i)
    
  # return segmented_docs,new_df
  return new_df

In [ ]:
sentences=df['text'].tolist()
# segmented_docs,corpus_split_df=segment_documents(df,sentences)
corpus_split_df=segment_documents(df,sentences)


In [ ]:
corpus_split_df.to_csv("corpus_split.csv")

### sentence_bleu

In [ ]:
# for q in tqdm(questions['data']):
#     test_question = q['question']
#     #Tokenize question in test data
#     tokens1 = ViTokenizer.tokenize(test_question.lower())
#     retrieval_result = []
#     for per_dictionary in data['data']:
#         if per_dictionary['category']=='FALSE_LONG_ANSWER':
#             continue
#         #Tokenize question in training data
#         tokens2 = ViTokenizer.tokenize(per_dictionary['question'].lower())
#         #Calculate score
#         score = sentence_bleu([remove_stopword(tokens1)], remove_stopword(tokens2), smoothing_function=SmoothingFunction().method4)
#         if score > 0:
#             per_dictionary['score'] = score
#             retrieval_result.append(per_dictionary)
#     if len(retrieval_result)==0:
#         continue
#     # Sort relevant question in training
#     retrieval_result = sorted(retrieval_result, key=lambda x: x['score'], reverse = True)
#     result = qa_model(question = test_question, context = retrieval_result[0]['text'])['answer']
#     q['category'] = 'T_F_LONG_ANSWER'
#     q['text']= retrieval_result[0]['text']
#     if is_date_format(result):
#         result = to_date_format(result)
#     elif has_numbers(result):
#         result = to_number_format(result)
#     else:
#         result = to_wiki_format(result)
#     q['answer'] = result


### esemble: util.semantic_search + sentence_bleu

In [ ]:
# top_k = 3
# for q in tqdm(questions['data']):
#     test_question = q['question']
#     #Tokenize question in test data
#     tokens1 = ViTokenizer.tokenize(test_question.lower())
#     retrieval_result=[]
#     for per_dictionary in data['data']:
#       tokens2 = ViTokenizer.tokenize(per_dictionary['question'].lower())
#       score = sentence_bleu([remove_stopword(tokens1)], remove_stopword(tokens2), smoothing_function=SmoothingFunction().method4)
#       if score > 0.8:
#           per_dictionary['score'] = score
#           retrieval_result.append(per_dictionary)
#     retrieval_result = sorted(retrieval_result, key=lambda x: x['score'], reverse = True)
#     # result = qa_model(question = test_question, context = retrieval_result[0]['text'])['answer']
    
#     if retrieval_result:
#       result= retrieval_result[0]['text']
#       if is_date_format(result):
#           result = to_date_format(result)
#       elif has_numbers(result):
#           result = to_number_format(result)
#       else:
#           result = to_wiki_format(result)
#       q['answer'] = result

#       continue

#     query_embedding = model.encode(test_question,convert_to_tensor=True,)
#     hits_question = util.semantic_search(query_embedding, small_corpus_question, top_k=3,score_function=util.cos_sim)[0]
#     hits_title = util.semantic_search(query_embedding, small_corpus_title, top_k=3,score_function=util.cos_sim)[0]
#     hits_text = util.semantic_search(query_embedding, small_corpus_text, top_k=3,score_function=util.cos_sim)[0]
#     question_answers =[]
#     title_answers =[]
#     text_answers =[]
    
#     for hit in hits_question:
#         corpus_id = hit['corpus_id']
#         # doc_score = hit['score']
#         title = data_fr.iloc[corpus_id]['title']
#         text =  data_fr.iloc[corpus_id]['text']
#         text = re.sub(r'\s+',' ',text)
#         text= re.sub(r'_',' ',text)
#         # question_answers.append((title,qa_model(question = test_question, context = text)['answer']))
#         question_answers.append((title,text))


#     for hit in hits_title:
#         corpus_id = hit['corpus_id']
#         # doc_score = hit['score']
#         title = data_fr.iloc[corpus_id]['title']
#         text =  data_fr.iloc[corpus_id]['text']
#         text = re.sub(r'\s+',' ',text)
#         text= re.sub(r'_',' ',text)
#         # title_answers.append((title,qa_model(question = test_question, context = text)['answer']))
#         title_answers.append((title,text))
#     for hit in hits_text:
#         corpus_id = hit['corpus_id']
#         # doc_score = hit['score']
#         title = data_fr.iloc[corpus_id]['title']
#         text =  data_fr.iloc[corpus_id]['text']
#         text = re.sub(r'\s+',' ',text)
#         text= re.sub(r'_',' ',text)
#         text_answers.append((title,qa_model(question = test_question, context = text)['answer']))
#         text_answers.append((title,text))
#     # Sort relevant question in training
#     q['text_answers']=text_answers
#     q['title_answers']=title_answers
#     q['question_answers']=question_answers

#     result=question_answers[0][1]
#     # sc.append(answers[0][0])
#     # result = sorted(answers, key=lambda x: x[1], reverse = True)
#     if is_date_format(result):
#         result = to_date_format(result)
#     elif has_numbers(result):
#         result = to_number_format(result)
#     else:
#         result = to_wiki_format(result)
#     q['answer'] = result


In [ ]:
# for q in questions['data']:
#     print(q['answer'])

In [ ]:
# json_object = json.dumps(questions, indent=4, ensure_ascii=False)
# with open("submission.json", "w", encoding='utf-8') as result:
#     result.write(json_object)

# TF_IDF

In [ ]:
# !gdown 1RmeuAiw7ZEDGJ71xR74f2kc0SIJBFtSQ
# !gdown 1UF51tO27w59RXRKt-7JcZ8uZxxntG_vN
# !gdown 1jrOTfb_87nroC12pDadzbIfSP8SjbkRk
# df_1=pd.read_csv("df_1.csv")
# df_2=pd.read_csv("df_2.csv")
# df_3=pd.read_csv("df_3.csv")
# df=pd.concat([df_1,df_2,df_3])

In [ ]:
def combine_metadata_wiki(item):
    newItem = {}
    # newItem['title'] = item['title']
    # newItem['text'] = item['text']
    newItem['text'] = item
    return newItem
def count_word_in_dataset_wiki(items):
    '''
    count number of word in dataset
    '''
    nameCounter = {}
    descCounter = {}
    for item in items:
        # for word in item['unigram_title'].keys():
        #     if word in nameCounter.keys():
        #         nameCounter[word] += 1
        #     else:
        #         nameCounter[word] = 1
        for word in item['unigram_text'].keys():
            if word in descCounter.keys():
                descCounter[word] += 1
            else:
                descCounter[word] = 1
    return  descCounter

def count_unigram(text):
    '''
    Count appearance number for each vocabulary
    '''
    counter = {}
    words = str(text).split()
    vocabs = set(words)
    for vocab in vocabs:
        if not vocab.isdigit():
            counter[vocab] = words.count(vocab)
    return counter


### Concat wiki corpus and question aim to transform tf-idf

In [ ]:

items=remove_stopword(corpus_split_df['text']).tolist()

In [ ]:
newItems_wiki = []
for item in tqdm(items):
    newItem = combine_metadata_wiki(item)
    # newItem['unigram_title'] = count_unigram(newItem['title'])
    newItem['unigram_text'] = count_unigram(newItem['text'])
    newItems_wiki.append(newItem)

  0%|          | 0/21457 [00:00<?, ?it/s]

In [ ]:
def tfidf(doc_len, corpus_len, doc_counter, corpus_counter, k=2):
    vector_len = len(corpus_counter)
    tfidf_vector = np.zeros((vector_len,))
    for i, key in enumerate(corpus_counter.keys()):
        if key in doc_counter.keys():
            tf = (k+1)*doc_counter[key]/(k+doc_counter[key])
            idf = math.log((corpus_len+1)/(corpus_counter[key]))
            tfidf_vector[i] = tf*idf
    return tfidf_vector

In [ ]:
avgdl = sum(len(sentence) for sentence in data['text']) / len(data['text'])
def bm25(doc_len, corpus_len, doc_counter, corpus_counter, k=1.8,b=0.75):
    vector_len = len(corpus_counter)
    tfidf_vector = np.zeros((vector_len,))
    for i, key in enumerate(corpus_counter.keys()):
        if key in doc_counter.keys():
            tf = (k+1)*doc_counter[key]/(doc_counter[key]+k*(1-b+b*doc_len/avgdl) )
            idf = math.log((corpus_len-corpus_counter[key]+0.5)/(corpus_counter[key]+0.5)+1)
            tfidf_vector[i] = tf*idf
    return tfidf_vector

In [ ]:
# !gdown 1MuLmncb4jjtuGeOuvS7JtCSnN8xReZ5w
# fn = "newItems_wiki.json"
# newItems_wiki = []
# with open(fn, "r") as f:
#     newItems_wiki = json.load(f)


# # print(len(newItems_wiki))
# test_item = newItems_wiki[0]
# pprint(test_item)
textCounter = count_word_in_dataset_wiki(newItems_wiki)

In [ ]:
tfidf_vectors = []
corpus_len = len(newItems_wiki)
for item in tqdm(newItems_wiki):
    doc_len = len(item['text'])
    tfidf_vectors.append(
        tfidf(doc_len, corpus_len, item['unigram_text'], textCounter)
    )


  0%|          | 0/21457 [00:00<?, ?it/s]

  0%|          | 0/21457 [00:00<?, ?it/s]

In [ ]:
tfidf_vectors = sparse.csr_matrix(tfidf_vectors)

# question_tfidf_vectors = tfidf_vectors[20857:]
# corpus_tfidf_vectors = tfidf_vectors[:20857]
# question_tfidf_vectors_question = tfidf_vectors_question[20857:]
# corpus_tfidf_vectors_question = tfidf_vectors_question[:20857]

In [ ]:
bm25_vectors = []
corpus_len = len(newItems_wiki)
for item in tqdm(newItems_wiki):
    doc_len = len(item['text'])
    bm25_vectors.append(
        bm25(doc_len, corpus_len, item['unigram_text'], textCounter)
    )

  0%|          | 0/21457 [00:00<?, ?it/s]

  0%|          | 0/21457 [00:00<?, ?it/s]

In [ ]:
bm25_vectors = sparse.csr_matrix(bm25_vectors)

# question_bm25_vectors = bm25_vectors[20857:]
# corpus_bm25_vectors = bm25_vectors[:20857]
# question_bm25_vectors_question = bm25_vectors_question[20857:]
# corpus_bm25_vectors_question = bm25_vectors_question[:20857]

In [ ]:
# query_vector = question_tfidf_vectors_question[4]
# query_vector = np.reshape(query_vector, (1,-1))
# sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_tfidf_vectors_question)
# sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
# idx = (-sim_maxtrix).argsort()[:5]
# for _id in idx:
#     print(_id, sim_maxtrix[_id])
#     print(newItems_wiki_question[_id]['title'].upper())
#     print(newItems_wiki_question[_id]['question'], "\n\n")

19737 0.9118133399322441
MUAMMAR AL-GADDAFI
Tổ chức thống nhất châu Phi thành lập năm nào 


4911 0.9118133399322441
MAROC
Tổ chức thống nhất châu Phi thành lập năm nào 


8043 0.7053357986947547
TỔ CHỨC CHÂU PHI THỐNG NHẤT
Tổ chức thống nhất châu Phi gồm bao nhiêu nước thành viên 


11716 0.7053357986947547
NGÀY TỊ NẠN THẾ GIỚI
Tổ chức thống nhất châu Phi gồm bao nhiêu nước thành viên 


1556 0.7053357986947547
CÚP BÓNG ĐÁ CHÂU PHI 1959
Tổ chức thống nhất châu Phi gồm bao nhiêu nước thành viên 




In [ ]:
# query_vector = question_tfidf_vectors[4]
# query_vector = np.reshape(query_vector, (1,-1))
# sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_tfidf_vectors)
# sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
# idx = (-sim_maxtrix).argsort()[:5]
# for _id in idx:
#     print(_id, sim_maxtrix[_id])
#     print(newItems_wiki[_id]['title'].upper())
#     print(newItems_wiki[_id]['text'], "\n\n")

6938 0.3024805885859133
IL CANTO DEGLI ITALIANI
Chiến thắng đã ở đâu rồi? 


8374 0.24796351963742497
AI CẬP
Ai Cập là một thành viên của Liên Hiệp Quốc , Phong trào không liên kết , Liên đoàn Ả Rập , Liên minh châu Phi , và Tổ chức Hợp tác Hồi giáo . 


8043 0.24534826457751874
TỔ CHỨC CHÂU PHI THỐNG NHẤT
Tổ chức châu Phi Thống nhất (OAU) (tiếng Pháp: Organization de l'Unité Africaine (OUA)) được thành lập ngày 25 tháng 5 năm 1963 tại Addis Ababa, với 32 quốc gia ký kết. Tổ chức này đã bị giải tán vào ngày 09 tháng 7 năm 2002 bởi Chủ tịch cuối cùng của nó, Tổng thống Nam Phi Thabo Mbeki, và được thay thế bởi Liên minh châu Phi (AU). 


11716 0.24501915080038417
NGÀY TỊ NẠN THẾ GIỚI
Trước 2000, Ngày tị nạn châu Phi đã được chính thức tổ chức ở nhiều nước vào ngày này. Liên Hiệp Quốc cho biết, Tổ chức Thống nhất châu Phi (OAU) đã đồng ý để ngày người tị nạn quốc tế trùng ngày với Ngày tị nạn châu Phi. 


4195 0.2403023060994857
NGÔ QUÂN MIỆN
Tưởng đâu sông Đuống ngủ rồi 




In [ ]:
import pandas as pd
# !gdown 1S1GXORdEP2nsFkPX6d_qzTu-Z4Q8RaAk
# df=pd.read_csv("wiki.csv")
df = pd.read_json('wikipedia_20220620_cleaned_parsed.jsonl', orient='records', lines=True)
df.drop(columns=['Unnamed: 0','id','url','timestamp','revid'],inplace=True)
wiki_corpus=df['text'].tolist()
df=[]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer_corpus = vectorizer.fit_transform(wiki_corpus)
# qqq__=[]
# for q in questions['data']:
#   qqq__.append(q['question'])
# qqq=vectorizer.transform(qqq__)

In [ ]:
for i in tqdm(range(600)):    
    if check[i]['answer'][1] > 0.999:
        questions['data'][i]['answer']=check[i]['answer'][0]
        questions['data'][i]['score'] = check[i]['answer'][1]
        continue
    text_answers =[]
    query_embedding = model_.encode(test_question,convert_to_tensor=True,)
    hits_text = util.semantic_search(query_embedding, small_corpus_text, top_k=3,score_function=util.cos_sim)[0]
    for hit in hits_text:
        corpus_id = hit['corpus_id']
        text =  wiki_corpus[corpus_id]
        text = re.sub(r'\s+',' ',text)
        text= re.sub(r'_',' ',text)
        text_answers.append(qa_model(question = test_question, context = text))

    




    query_vector = question_bm25_vectors_question[i]
    sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_bm25_vectors_question)
    sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
    idx = (-sim_maxtrix).argsort()[:10]
    score_list=[]
    for _id in idx:
        result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
        score_list.append((result['answer'],result['score']))
    score_list = sorted(score_list, key=lambda x: x[1], reverse = True)
    questions['data'][i]['score'] = score_list[0][1]
    result = score_list[0][0]
    if is_date_format(result):
        result = to_date_format(result)
    elif has_numbers(result):
        result = to_number_format(result)
    else:
        result = to_wiki_format(result)
    questions['data'][i]['answer']=result


In [ ]:
for i in tqdm(range(600)):   
    if check[i]['answer'][1] > 0.997:
        questions['data'][i]['answer'] =  check[i]['answer'][0]
        continue    
    query_vector = question_bm25_vectors_question[i]
    query_vector = np.reshape(query_vector, (1,-1))
    sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_bm25_vectors_question)
    sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
    idx = (-sim_maxtrix).argsort()[:10]
    result=None
    index_result_score=None
    score=[]
    doc=[]
    question_result_list=[]
    question_result_list_score=[]
    result_list=[]
    result_list_score=[]
    for _id in idx:
        # result = newItems_wiki_question[_id]['question']
        # result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
        # result_ans = result['answer']
        # result = data['data'][_id]['text']
        # score = sim_maxtrix[_id]
        # questions['data'][i]['doc_score'] = score
        # questions['data'][i]['question_retrieval'] = data['data'][_id]['question']
        result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
        question_result_list.append(result['answer'])
        question_result_list_score.append(result['score'])
        score.append(sim_maxtrix[_id])
        doc.append(data['data'][_id]['question'])
    # questions['data'][i]['question_answer'] = doc
    # questions['data'][i]['doc_question_score'] = score
    # questions['data'][i]['question_result'] = result_list
    # questions['data'][i]['question_result_score'] = result_list_score
    index_question_score = np.array(question_result_list_score).argsort()[-1]
    if  data['data'][index_question_score]['category']=="FALSE_LONG_ANSWER":
        # score=[]
        # doc=[]
        
        query_vector = question_bm25_vectors[i]
        query_vector = np.reshape(query_vector, (1,-1))
        sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_bm25_vectors)
        sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
        idx = (-sim_maxtrix).argsort()[:10]
        for _id in idx:                   
            # result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
            # result = data['data'][_id]['text']
            # result_ans = result['answer']
            # questions['data'][i]['doc_score'] = sim_maxtrix[_id]
            # questions_file['data'][i]['ans_score'] = result['score']
            result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
            result_list.append(result['answer'])
            result_list_score.append(result['score'])
            score.append(sim_maxtrix[_id])
            doc.append(data['data'][_id]['text'])
        index_result_score = np.array(result_list_score).argsort()[-1]


    # if question_result_list_score[index_question_score] > result_list_score[index_result_score]:
    #     result = question_result_list[index_question_score]
    # else:
    #     result = question_result_list[index_result_score]

    # questions['data'][i]['ans_score'] = result['score']
    # result = result['answer']
    question_result = question_result_list[index_question_score]
    result = question_result
    if result_list != []:  
      if question_result_list_score[index_question_score] < result_list_score[index_result_score]:
        result = result_list[index_result_score]
    
    if is_date_format(result):
        result = to_date_format(result)
    elif has_numbers(result):
        result = to_number_format(result)
    else:
        result = to_wiki_format(result)
    # questions_file['data'][i]['result'] = result
    
    # if is_date_format(question_result):
    #     question_result = to_date_format(question_result)
    # elif has_numbers(question_result):
    #     question_result = to_number_format(question_result)
    # else:
    #     question_result = to_wiki_format(question_result)

    questions['data'][i]['answer'] = result
    # questions['data'][i]['question_answer'] = question_result

    # questions['data'][i]['answer']=doc
    # questions['data'][i]['doc_score'] = score
    # questions['data'][i]['result'] = result_list
    # questions['data'][i]['result_score'] = result_list_score


  0%|          | 0/600 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1046: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


In [ ]:
for i in tqdm(range(600)):    
    query_vector = question_tfidf_vectors_question[i]
    query_vector = np.reshape(query_vector, (1,-1))
    sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_tfidf_vectors_question)
    sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
    idx = (-sim_maxtrix).argsort()[:5]
    result=None
    # score=None
    score=[]
    doc=[]
    for _id in idx:
        # result = newItems_wiki_question[_id]['question']
        # result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
        # result_ans = result['answer']
        # score = sim_maxtrix[_id]
        score.append(sim_maxtrix[_id])
        # questions['data'][i]['question_retrieval'] = data['data'][_id]['question']
        doc.append(data['data'][_id]['question'])
    questions['data'][i]['doc_question_score'] = score
    if max(score) < 0.7 or data['data'][_id]['is_long_answer']!="true":
        query_vector = question_tfidf_vectors[i]
        query_vector = np.reshape(query_vector, (1,-1))
        sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, corpus_tfidf_vectors)
        sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
        idx = (-sim_maxtrix).argsort()[:5]
        for _id in idx:                   
            # result = qa_model(question = questions['data'][i]['question'], context = data['data'][_id]['text'])
            # result = data['data'][_id]['text']
            # result_ans = result['answer']
            # questions['data'][i]['doc_score'] = sim_maxtrix[_id]
            score.append(sim_maxtrix[_id])
            
            doc.append(data['data'][_id]['text'])
            # questions_file['data'][i]['ans_score'] = result['score']
    # questions['data'][i]['ans_score'] = result['score']
    # result = result['answer']

    # if is_date_format(result):
    #     result = to_date_format(result)
    # elif has_numbers(result):
    #     result = to_number_format(result)
    # else:
    #     result = to_wiki_format(result)
    # questions_file['data'][i]['result'] = result
    
    # questions['data'][i]['answer'] = result
    questions['data'][i]['answer']=doc
    questions['data'][i]['doc_score'] = score

  0%|          | 0/600 [00:00<?, ?it/s]

In [ ]:
json_object = json.dumps(questions, indent=4, ensure_ascii=False)
with open("submission.json", "w", encoding='utf-8') as result:
    result.write(json_object)

In [ ]:
query_vector = tfidf_vectors[0]
query_vector = np.reshape(query_vector, (1,-1))
# search
sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, tfidf_vectors)
sim_maxtrix = np.reshape(sim_maxtrix, (-1,))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(da)

#####

In [ ]:
ques=["Hiến pháp lâu đời nhất thế giới"]
query_vector=vectorizer.transform(ques)

In [ ]:
ques=[questions_file['data'][2]['question']]
query_vector=vectorizer.transform(ques)

In [ ]:
sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, X)
sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
idx = (-sim_maxtrix).argsort()[:5]
print(ques)
for _id in idx:
    print(_id, sim_maxtrix[_id])
    print(newItems_wiki[_id]['title'].upper())
    print(newItems_wiki[_id]['text'], "\n\n")

In [ ]:
for q in tqdm(questions_file['data']):
    test_question = q['question']
    #Tokenize question in test data
    tokens1 = ViTokenizer.tokenize(test_question.lower())
    retrieval_result=[]
    for per_dictionary in data['data']:
      tokens2 = ViTokenizer.tokenize(per_dictionary['question'].lower())
      score = sentence_bleu([remove_stopword(tokens1)], remove_stopword(tokens2), smoothing_function=SmoothingFunction().method4)
      if score < 0.75 or score==1:
          per_dictionary['score'] = score
          retrieval_result.append(per_dictionary)
    retrieval_result = sorted(retrieval_result, key=lambda x: x['score'], reverse = True)
    
    
    if retrieval_result:
      result = qa_model(question = test_question, context = retrieval_result[0]['text'])['answer']
      if is_date_format(result):
          result = to_date_format(result)
      elif has_numbers(result):
          result = to_number_format(result)
      else:
          result = to_wiki_format(result)
      q['answer'] = result

      continue
    #Tokenize question in test data
    # tokens1 = ViTokenizer.tokenize(test_question.lower())
    query_vector=vectorizer.transform([test_question])
    sim_maxtrix = sklearn.metrics.pairwise.cosine_similarity(query_vector, X )
    sim_maxtrix = np.reshape(sim_maxtrix, (-1,))
    result=None
    idx = (-sim_maxtrix).argsort()[:1]
    for _id in idx:
        # print(_id, sim_maxtrix[_id])
        # print(newItems_wiki[_id]['title'].upper())
        # title_retrieval.append(newItems_wiki[_id]['title'])
        # retrieval_result.append(newItems_wiki[_id]['text'])
      result = qa_model(question = test_question, context = newItems_wiki[_id]['text'])['answer']

    if is_date_format(result):
        result = to_date_format(result)
    elif has_numbers(result):
        result = to_number_format(result)
    else:
        result = to_wiki_format(result)
    q['answer'] = result

In [ ]:
json_object = json.dumps(questions_file, indent=4, ensure_ascii=False)
with open("submission.json", "w", encoding='utf-8') as result:
    result.write(json_object)